In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from sklearn.metrics.pairwise import cosine_similarity
from io import StringIO
import csv
import torch
import heapq
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM
)


from tqdm import tqdm
import numpy as np
import pandas as pd

data = pd.read_csv('../data/pseudo_json_dataset_V3_2048.csv')

/mnt/gime-extract/miniconda3/envs/llmenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_id = 'RUCKBReasoning/TableLLM-7b'
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [3]:
index_df = pd.read_csv('../data/index_info.csv')
index_df.head()

,Index of data without headers,Index of data with multiple headers,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,1,692.0,NaN,NaN,
1,36,712.0,NaN,NaN,NaN
2,60,NaN,NaN,NaN,NaN
3,90,NaN,NaN,NaN,NaN
4,111,NaN,NaN,NaN,NaN


In [4]:
model = AutoModelForCausalLM.from_pretrained(
    model_id, device_map="auto", torch_dtype=torch.float16,
)

Loading checkpoint shards: 100%|██████████| 3/3 [02:37<00:00, 52.64s/it]


In [5]:
index_info = index_df['Index of data without headers'].to_list()

In [6]:
device = torch.device('cuda:0')

In [7]:
def convert_to_embeddings(text):
    tokens = tokenizer.tokenize(text)
    token_id = tokenizer.convert_tokens_to_ids(tokens)
    embeddings = model.model.embed_tokens(torch.tensor(token_id).to(device))
    return embeddings.to('cpu').detach().numpy()

In [14]:
output_df = pd.DataFrame(columns=['index', 'question', 'csv_row', 'cosine_similarity'])

In [15]:
# with open(filename, 'w') as file:
count = 0
for i, row in tqdm(data[:2000].iterrows()):
    soup = BeautifulSoup(row['html_content'], 'html.parser')

    # Find the table
    table = soup.find('table')
    # Open StringIO object for writing
    csv_buffer = StringIO()


    trows = table.find_all('tr')
    question_embed = convert_to_embeddings(row['question'])
    
    if row['index'] not in index_info: 
        header = trows[0]

        for trow in trows[1:]:
            writer = csv.writer(csv_buffer)
            writer.writerow([cell.get_text() for cell in trow.find_all(['td', 'th'])])
            csv_data = csv_buffer.getvalue().replace('\n', '').replace('  ', '')
            csv_data_embed = convert_to_embeddings(csv_data)
            csv_buffer = StringIO()
            cosine_sim = cosine_similarity(csv_data_embed, question_embed)
            cosine_sim = cosine_sim.sum() / ( cosine_sim.shape[0] * cosine_sim.shape[1] )
            output_df.at[count, 'question'] = row['question']
            output_df.at[count, 'cosine_similarity'] = cosine_sim
            output_df.at[count, 'csv_row'] = csv_data
            output_df.at[count, 'index'] = row['index']
            count+=1
    else:
        for trow in trows:
            writer = csv.writer(csv_buffer)
            writer.writerow([cell.get_text() for cell in trow.find_all(['td', 'th'])])
            csv_data = csv_buffer.getvalue().replace('\n', '').replace('  ', '')
            csv_data_embed = convert_to_embeddings(csv_data)
            csv_buffer = StringIO()
            cosine_sim = cosine_similarity(csv_data_embed, question_embed)
            cosine_sim = cosine_sim.sum() / ( cosine_sim.shape[0] * cosine_sim.shape[1] )
            output_df.at[count, 'question'] = row['question']
            output_df.at[count, 'cosine_similarity'] = cosine_sim
            output_df.at[count, 'csv_row'] = csv_data
            output_df.at[count, 'index'] = row['index']
            count+=1
        
        # writer.writerow([cell.get_text() for cell in trow.find_all(['td', 'th'])])


    # Get the CSV data as a string
    

2000it [00:41, 48.27it/s]


In [16]:
output_df.to_csv('../embedding_similarity/table7B_LLM.csv', index=False)

: 